In [4]:
import os
from fyenn_class import  pd_loaddata, pd, np 
from datetime import datetime, timedelta

path = '../data_down/'
os.listdir(path)

['gar.xlsx',
 'huawei_daily.csv',
 'Outsourcing_Name_List_20220119104910.csv',
 'Outsourcing_Name_List_20220119104910.xlsx',
 '业务量与部门关系表_20220119104837.csv',
 '业务量与部门关系表_20220119104837.xlsx',
 '入湖数据关系.csv',
 '入湖数据关系.xlsx']

In [15]:
# '{note:=>50}'.format(note='Lorem') + '{note:=>50}'.format(note='')
# huawei_output = huawei_output[huawei_output['rn'].astype(int) == 1]

# rel 
huawei_output = pd_loaddata.pd_csv(path + 'huawei_daily.csv')
huawei_output = pd_loaddata.bdp_col(huawei_output)


In [16]:
col = 'update_date'
huawei_output[col] = huawei_output[col].astype(str).str.slice(0,10)
huawei_output = pd.concat([huawei_output.reset_index(), pd.DataFrame(list(huawei_output[col].str.split('-')))], axis =1)

In [56]:
def printer(*args):
    [print( '{note:~>25}'.format(note = i)) for i in args]

printer(col, col)

~~~~~~~~~~~~~~update_date
~~~~~~~~~~~~~~update_date


[None, None]

In [19]:
%%time
# huawei_output = pd_loaddata.pd_csv(path + 'huawei_output.csv')
# huawei_output = huawei_output[huawei_output['rn'].astype(int) == 1]

# huawei_daily = pd_loaddata.pd_csv(path + 'huawei_daily.csv')
# rel = pd_loaddata.pd_excel(path + '入湖数据关系', 0)
# huawei_output.shape
# pd_loaddata.pd_show(15, None)
# rel = rel.drop(['部门', '站点'],axis = 1).drop_duplicates();rel


def datetime_(coach, col):
        """
        detail time to date; 
        split and assign to 3 cols of year m d
        """
        coach[col] = coach[col].astype(str).str.slice(0,10)
        # 20220110
        coach = pd.concat([coach.reset_index(drop = True), pd.DataFrame(list(coach[col].str.split('-')))], axis =1) 
        coach = coach[coach[[0,1,2]].fillna(0).astype(int).sum(axis = 1) != 0] # 年月日三列
        coach = coach.rename({0:'year', 1:'month', 2: 'date'}, axis=1)
        return coach

huawei_output = datetime_(
    coach=huawei_output, col='update_date'
    ).drop(['create_time', 'update_time'], axis = 1).drop_duplicates().sort_values('update_date')
 
def search_col(df, str):
    """
    列名正则搜索.
    """
    return list(pd.Series(df.columns)[pd.Series(df.columns).str.match(str)])
# 识别异常列名
tran_col = list(pd.Series(huawei_output.columns)[
    pd.Series(huawei_output.columns).str.contains(
        '(receive|psn|send|transport_times|update_date|year|month|date|id)'
        ) == False
        ])
# clean , case, withdraw)

Wall time: 553 ms


<timed exec>:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [20]:
pd.Series(huawei_output.columns)[pd.Series(huawei_output.columns).str.contains('t4\_')]

Series([], dtype: object)

In [27]:
# unfigured problems:::::

huawei_output = huawei_output.iloc[:, :-3]

In [28]:
huawei_output['te_origin_receive'] = huawei_output['te_origin_receive'] + huawei_output['te_product_receive'] 
huawei_output['te_origin_send'] = huawei_output['te_origin_send'] + huawei_output['te_product_send'] 
huawei_output = huawei_output.drop(['te_product_send', 'te_product_receive'], axis = 1)

In [29]:

"""
ou 和正则匹配
"""
relist = [
        'origi', 
        'hon',  # 20220119 to hongmei
        'pearl', 
        'guiy', 
        '^t\_', 
        'r4\_', 
        'nanh', 
        'ansh', 
        'te\_',
        'pingsha' # 20220119 add
      
    ]

oulist = [
        'HUAWEDHW4S',
        'HONORDGHMS',
        'HUAWEDHWTS',
        'HUAWEDGTRD',
        'HUAWEDGLSS',
        'HUAWEDHW1S',
        'HUAWEDGNHS',
        'NEXPEDGWHS',
        'TYCOTSDXXS',
        'HONORSZIHS'  # 20220119 add
    ]
try:
    del dict 
except:
    pass
dict = dict(zip(relist, oulist))
# re = 'pear'
# pd.Series(huawei_output.columns).str.extract("(" + re + "[a-z]+)").dropna()
dict

{'origi': 'HUAWEDHW4S',
 'hon': 'HONORDGHMS',
 'pearl': 'HUAWEDHWTS',
 'guiy': 'HUAWEDGTRD',
 '^t\\_': 'HUAWEDGLSS',
 'r4\\_': 'HUAWEDHW1S',
 'nanh': 'HUAWEDGNHS',
 'ansh': 'NEXPEDGWHS',
 'te\\_': 'TYCOTSDXXS',
 'pingsha': 'HONORSZIHS'}

In [30]:
huawei_output = huawei_output.drop('index', axis = 1)
huawei_output.columns

Index(['update_date', 'origin_receive', 'origin_send', 'origin_psn',
       'hongmei_receive', 'pingshan_receive', 'hongmei_send', 'pingshan_send',
       'hongmei_psn', 'pearlriver_transport_times', 'guiyang_transport_times',
       't_product_receive', 't_product_clean', 't_product_send', 'r4_receive',
       'r4_send', 'r4_psn', 'nanhua_receive', 'nanhua_send', 'nanhua_psn',
       'anshi_receive', 'anshi_send', 'anshi_case', 'te_origin_receive',
       'te_origin_send', 'pre_receive', 'pre_withdraw',
       'honor_transport_times', 'inc_day', 'year', 'month', 'date'],
      dtype='object')

In [ ]:
pd.concat([huawei_output[search_col(huawei_output, 'hon')] ,
 huawei_output[['update_date', 'year', 'month', 'date']]], axis = 1)

In [31]:
def concat_(re, ou):
    huawei_output[search_col(huawei_output, re)].shape
    # n = 4 - huawei_output[search_col(huawei_output, re)].shape[1]
    # m = huawei_output[search_col(huawei_output, re)].shape[0]
    # print(m, n)
    data = pd.concat(
        [
            huawei_output[search_col(huawei_output, re)], 
            # pd.DataFrame(np.zeros(shape=(m, n), dtype=int)),
            huawei_output[['update_date', 'year', 'month', 'date']]
        ]
        , axis = 1).sort_values('update_date')
    # print(data.shape)
    data.columns = list(pd.Series(data.columns).str.extract(
        '(receive|psn|send|transport_times|update_date|year|month|date)'
        )[0])
    try:
        print(data.columns, '\n', data.head(2))
        data = data.rename({np.nan:'addition'}, axis = 1)
    except:
        pass
    data['ou'] = ou
    data['station'] = pd.Series(huawei_output.columns).str.extract("(" + re + "[a-z]+)").dropna().iloc[0,0]
    data['addition_type'] = pd.Series(tran_col).str.extract("(" + re + ".+)").fillna('z').sort_values(0).iloc[0,0]
     
    return data.reset_index(drop=True)

df = pd.DataFrame()
for re in dict:
    print(re, dict[re])
    df = pd.concat([df, concat_(re, dict[re])], axis = 0)


origi HUAWEDHW4S
Index(['receive', 'send', 'psn', 'update_date', 'year', 'month', 'date'], dtype='object') 
        receive  send  psn update_date  year month date
14505      0.0   0.0  NaN  2020-12-31  2020    12   31
2345       0.0   0.0  NaN  2020-12-31  2020    12   31
22016      0.0   0.0  NaN  2020-12-31  2020    12   31
6692       0.0   0.0  NaN  2020-12-31  2020    12   31
10182      0.0   0.0  NaN  2020-12-31  2020    12   31
hon HONORDGHMS
Index(['receive', 'send', 'psn', 'transport_times', 'update_date', 'year',
       'month', 'date'],
      dtype='object') 
        receive  send  psn  transport_times update_date  year month date
14505      0.0   0.0  NaN              NaN  2020-12-31  2020    12   31
2345       0.0   0.0  NaN              NaN  2020-12-31  2020    12   31
22016      0.0   0.0  NaN              NaN  2020-12-31  2020    12   31
6692       0.0   0.0  NaN              NaN  2020-12-31  2020    12   31
10182      0.0   0.0  NaN              NaN  2020-12-31  2020  

In [32]:
pd.Series(tran_col).str.extract("(" + 'pear' + ".+)").fillna('zull')
# pd.Series(huawei_output.columns).str.extract("(" + 't_' + ".+)").dropna().iloc[0,0]
pd.Series(tran_col).str.extract("(" + 'anshi' + ".+)").fillna('z').sort_values(0).iloc[0,0]

'anshi_case'

In [33]:
def cleanm(df):
    """
    清楚每日填报重复, 时间日期drop, update_date 日期加一以匹配前端.
    """
    df = df.fillna(0).drop_duplicates()
    df['flag_sum'] = df[['receive', 'send', 'psn']].sum(axis =1)
    df = df.sort_values(['update_date', 'ou', 'flag_sum'], ascending=False).groupby(
        [
            'update_date', 'ou'
        ]
        ).first().reset_index()
    try:
        df = df.drop(['year', 'month', 'date', 'flag_sum'], axis = 1)
    except:
        pass

    df['update_date'] = pd.to_datetime(df['update_date']) + timedelta(days = 1)
    df['addition_type'] = df['addition_type'].str.replace('^z', 'None')
    return df

df = cleanm(df)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_22920/680750927.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['addition_type'] = df['addition_type'].str.replace('^z', 'None')


In [ ]:
df[['receive', 'send', 'psn', 'transport_times', 'addition']] = df[
        ['receive', 'send', 'psn', 'transport_times', 'addition']
        ].astype(int)

In [ ]:
df.info()

In [ ]:
df.query("update_date == '2021-11-27'")

In [ ]:
list(pd.Series(huawei_output.columns).str.extract("(" + 'pearl' + "[a-z]+)").dropna()[0])[0]

In [ ]:
print('duplicates check',df[df[['update_date', 'ou']].duplicated()].sort_values(['update_date', 'ou']))